### Production Features Pipeline

In [1]:
# select web scraper; 'SCRAPINGANT' or 'SELENIUM'
# SCRAPINGANT requires a subscription but includes a proxy server

WEBSCRAPER = 'SCRAPINGANT'
# WEBSCRAPER = "SELENIUM"

In [2]:
import os

import pandas as pd
import numpy as np

import hopsworks

from datetime import datetime, timedelta
from pytz import timezone

import json

import time

from pathlib import Path  # for Windows/Linux compatibility

# change working directory to project root when running from notebooks folder to make it easier to import modules
# and to access sibling folders
os.chdir("..")


from src.utils.webscraping import (
    get_new_games,
    activate_web_driver,
    get_todays_matchups,
)

from src.data.cleaning import (
    process_games,
    add_TARGET,
)

from src.data.build_features import (
    process_features,
)

from src.utils.hopsworks_utils import (
    save_feature_names,
    convert_feature_names,
)

from src.utils.constants import (
    FEATURE_GROUP_VERSION,
)

DATAPATH = Path(r"data")

**Load API keys**

In [3]:
from dotenv import load_dotenv

load_dotenv()

try:
    HOPSWORKS_API_KEY = os.environ["HOPSWORKS_API_KEY"]
except:
    raise Exception("Set environment variable HOPSWORKS_API_KEY")

# if scrapingant is chosen then set the api key, otherwise load the selenium webdriver
if WEBSCRAPER == "SCRAPINGANT":
    try:
        SCRAPINGANT_API_KEY = os.environ["SCRAPINGANT_API_KEY"]
    except:
        raise Exception("Set environment variable SCRAPINGANT_API_KEY")
    driver = None

elif WEBSCRAPER == "SELENIUM":
    driver = activate_web_driver()
    SCRAPINGANT_API_KEY = ""

print(SCRAPINGANT_API_KEY[:5])

75f65


**Scrape New Completed Games and Format Them**

In [4]:
df_new = get_new_games(SCRAPINGANT_API_KEY, driver)

if df_new.empty:
    print("No new games to process")
else:

    # get the SEASON of the last game in the database
    # this will used when constructing rows for prediction
    SEASON = df_new["SEASON"].max()

print(df_new.head())

Current month is 03
Scraping https://www.nba.com/stats/teams/boxscores?SeasonType=Regular+Season&DateFrom=03/15/25&DateTo=03/22/25
USING SCARAPER API


  GAME_DATE_EST  HOME_TEAM_WINS  PTS_home  FG_PCT_home  FG3_PCT_home  \
0    2025-03-21               1       128         54.0          40.0   
1    2025-03-21               1       128         57.7          50.0   
2    2025-03-21               1       123         52.2          45.2   
3    2025-03-21               0        99         45.2          40.0   
4    2025-03-21               1       123         52.5          35.3   

   FT_PCT_home  REB_home  AST_home  HOME_TEAM_ID   GAME_ID  PTS_away  \
0         80.0        38        30    1610612757  22401022       109   
1         80.8        43        29    1610612746  22401023       108   
2        100.0        46        33    1610612756  22401021       112   
3         75.0        38        24    1610612762  22401020       121   
4         75.0        44        25    1610612742  22401019       117   

   FG_PCT_away  FG3_PCT_away  FT_PCT_away  REB_away  AST_away  \
0         50.0          34.6         85.7        31        24   
1   